In [ ]:
import torch
from KFoldCV import KFoldCV
from torchmetrics.classification import BinaryMatthewsCorrCoef
from Model import Model
from PMDataset import PMDataset, DatasetPredict
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [ ]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import pickle
with open('BEST_MODEL_PARAMETERS_study_1.pickle', 'rb') as f:
    x= pickle.load(f)
print(x.params)


In [ ]:
BATCH_SIZE=128
NUM_BLOCKS= x.params['num_blocks']
WIDTH=x.params['width']
INPUT_SIZE=377
OUTPUT_SIZE=1
LR= x.params['lr']
BETAS= (x.params['beta_1'],x.params['beta_2'])
SAVE_PATH="Models/LastModel"
MODEL_NAME="LastModel"

In [ ]:
model = Model
dataset = PMDataset("PreProcessing/X_train.npy","PreProcessing/y_train.npy")
metric = BinaryMatthewsCorrCoef().to(device)
lossFn = BCEWithLogitsLoss()
optimizer = AdamW

In [ ]:
kf = KFoldCV(
    k=5,
    batchSize=128,
    metric=metric,
    savePath=SAVE_PATH,
    modelName=MODEL_NAME,
    modelClass=model,
    device=device,
    dataset=dataset,
    optimizerClass=optimizer,
    lossFn=lossFn,
    optimizerArgs=[LR,BETAS],
    earlyStoppingArgs=[1e-6,10,False],
    modelArgs=[INPUT_SIZE,OUTPUT_SIZE,NUM_BLOCKS,WIDTH,0.2],
)

In [ ]:
kf.crossValidate()

In [ ]:
print(kf.validScores)
print(sum(kf.validScores)/5)

In [9]:
dataset_test = ICSDatasetPredict("PreProcessing/X_test.npy")
test_data_loader= DataLoader(dataset_test,shuffle=False)

NameError: name 'ICSDatasetPredict' is not defined

In [ ]:
def predict(
    model: torch.nn.Module,
    dataLoader: torch.utils.data.DataLoader,
    device: torch.device,
):
    """
    This function computes the following evaluation metrics on a given dataset: Accuracy, recall, confusion matrix, 1 vs rest confusion matrix.

    Parameters
    ----------
    model: The model to evaluate.

    dataLoader: Dataloader for the dataset to evaluate the model.

    resultSavePath: path to where the results shoul be saved.

    name of the results file.

    device: Device to run the model on.

    """
    
    arr = torch.Tensor().to(device)

    model.eval()
    with torch.inference_mode():
        for batch, X in tqdm(enumerate(dataLoader)):
            X = X.to(device)

            y_pred = model(X)
            arr = torch.cat((arr, y_pred), 0)

    return pd.Series(torch.squeeze(arr, 1).cpu().numpy().astype(np.float32)).apply(lambda x: 'p' if x>=0 else 'e')

In [ ]:
for i in tqdm(range(0,9)):
    model= Model(INPUT_SIZE,OUTPUT_SIZE,NUM_BLOCKS,WIDTH).to(device)
    model.load_state_dict(torch.load(f"{SAVE_PATH}/{MODEL_NAME}_FOLD_{i}.pth"))
    predict(model=model,dataLoader=test_data_loader,device=device).to_csv(f'{SAVE_PATH}/{MODEL_NAME}_FOLD_{i}.csv')
    
    